# 创建自定义环境

本页面简要介绍了如何使用 Gymnasium 创建自定义环境，有关[更完整的教程](https://gymnasium.farama.org/tutorials/gymnasium_basics/environment_creation/)和渲染，请在阅读本页之前阅读[基本用法](basic)。

我们将实现一个非常简单的游戏，称为 GridWorldEnv，由固定大小的二维方格组成。智能体可以在每个时间步中垂直或水平移动网格单元，智能体的目标是导航到在情节开始时随机放置的目标所在的网格单元。

**游戏基本信息**

- 观察值提供目标和智能体的位置。
- 我们的环境下有4个离散动作，分别对应“右”、“上”、“左”和“下”的移动。
- 当智能体导航到目标所在的网格单元时，环境结束（终止）。
- 智能体只有在到达目标时才会获得奖励，即当智能体到达目标时奖励为1，否则为0。

## 环境 `__init__`

像所有环境一样，我们的自定义环境将继承自 {class}`gymnasium.Env`，它定义了环境的结构。环境的一个要求是定义观察值和动作空间，这些声明了环境的一般可能输入（动作）和输出（观察值）。如我们关于游戏的基本信息所述，我们的智能体有四个离散动作，因此我们将使用 `Discrete(4)` 空间，包含四个选项。

对于我们的观察值，有几个选项，在这个教程中，我们将假设我们的观察值看起来像 `{"agent": array([1, 0]), "target": array([0, 3])}`，其中数组元素代表智能体或目标的 `x` 和 `y` 位置。表示观察值的其他选项包括作为2d网格，其中的值代表网格上的智能体和目标，或者作为3d网格，每个“层”只包含智能体或目标信息。因此，我们将声明观察值空间为 {class}`~gymnasium.spaces.Dict`，智能体和目标空间是 {class}`~gymnasium.spaces.Box`，允许 `int` 类型的数组输出。

要查看可用于环境的完整空间列表，请参阅 [spaces](https://gymnasium.farama.org/api/spaces/)。

In [1]:
from typing import Optional
import numpy as np
import gymnasium as gym


class GridWorldEnv(gym.Env):

    def __init__(self, size: int = 5):
        # The size of the square grid
        self.size = size

        # Define the agent and target location; randomly chosen in `reset` and updated in `step`
        self._agent_location = np.array([-1, -1], dtype=np.int32)
        self._target_location = np.array([-1, -1], dtype=np.int32)

        # Observations are dictionaries with the agent's and the target's location.
        # Each location is encoded as an element of {0, ..., `size`-1}^2
        self.observation_space = gym.spaces.Dict(
            {
                "agent": gym.spaces.Box(0, size - 1, shape=(2,), dtype=int),
                "target": gym.spaces.Box(0, size - 1, shape=(2,), dtype=int),
            }
        )

        # We have 4 actions, corresponding to "right", "up", "left", "down"
        self.action_space = gym.spaces.Discrete(4)
        # Dictionary maps the abstract actions to the directions on the grid
        self._action_to_direction = {
            0: np.array([1, 0]),  # right
            1: np.array([0, 1]),  # up
            2: np.array([-1, 0]),  # left
            3: np.array([0, -1]),  # down
        }

## 构建观测值

由于我们需要在 {meth}`Env.reset <gymnasium.Env.reset>` 和 {meth}`Env.step <gymnasium.Env.step>` 中计算观测值，通常方便的做法是有 `_get_obs` 方法，将环境的状态转换为观测值。然而，这不是强制性的，你也可以分别在 {meth}`Env.reset <gymnasium.Env.reset>` 和 {meth}`Env.step <gymnasium.Env.step>` 中计算观察值。

```python
    def _get_obs(self):
        return {"agent": self._agent_location, "target": self._target_location}
```

还可以为 {meth}`Env.reset <gymnasium.Env.reset>` 和 {meth}`Env.step <gymnasium.Env.step>` 返回的辅助信息实现类似的方法。在我们的情况下，我们希望提供智能体与目标之间的曼哈顿距离：

```python
def _get_info(self):
        return {
            "distance": np.linalg.norm(
                self._agent_location - self._target_location, ord=1
            )
        }
```

通常，`info` 还会包含一些仅在 {meth}`Env.step <gymnasium.Env.step>` 方法内可用的数据（例如，单个奖励项）。在这种情况下，我们将不得不更新 `_get_info` 在  {meth}`Env.step <gymnasium.Env.step>` 中返回的字典。

## `reset` 函数

{meth}`Env.reset <gymnasium.Env.reset>`  的目的是为环境启动新回合，它有两个参数：`seed` 和 `options`。`seed` 可以用来将随机数生成器初始化为一个确定的状态，而 `options` 可以用来指定在 {meth}`Env.reset <gymnasium.Env.reset>` 中使用的值。在 {meth}`Env.reset <gymnasium.Env.reset>` 的第一行，你需要调用 `super().reset(seed=seed)`，这将初始化随机数生成器（{meth}`gymnasium.Env.np_random`）以供后续的 {meth}`Env.reset <gymnasium.Env.reset>` 使用。

在我们的自定义环境中，{meth}`Env.reset <gymnasium.Env.reset>` 需要随机选择代理和目标的位置（如果它们位置相同，则需要重复此操作）。{meth}`Env.reset <gymnasium.Env.reset>` 的返回类型是元组，包含初始观测值和任何辅助信息。因此，我们可以使用之前实现的 `_get_obs` 和 `_get_info` 方法来实现这一点。

## Step 函数

{meth}`Env.step <gymnasium.Env.step>` 方法通常包含环境的主要逻辑，它接受动作并计算应用该动作后的环境状态，返回一个元组，包括下一个观察值、结果奖励、环境是否终止、环境是否截断以及辅助信息。

对于我们的环境，在 {meth}`Env.step <gymnasium.Env.step>` 函数期间需要发生几件事情：

```{note}
- 使用 `self._action_to_direction` 将离散动作（例如，2）转换为网格方向和我们的智能体位置。为了防止智能体超出网格边界，我们将智能体的位置裁剪到边界内。
- 通过检查智能体当前位置是否等于目标位置来计算智能体的奖励。
- 由于环境在内部没有截断（我们可以在 {func}`make <gymnasium.make>` 期间应用时间限制包装器到环境），我们永久设置 `truncated` 为 `False`。
- 再次使用 `_get_obs` 和 `_get_info` 来获取智能体的观察值和辅助信息。
```

```python
    def step(self, action):
        # Map the action (element of {0,1,2,3}) to the direction we walk in
        direction = self._action_to_direction[action]
        # We use `np.clip` to make sure we don't leave the grid bounds
        self._agent_location = np.clip(
            self._agent_location + direction, 0, self.size - 1
        )

        # An environment is completed if and only if the agent has reached the target
        terminated = np.array_equal(self._agent_location, self._target_location)
        truncated = False
        reward = 1 if terminated else 0  # the agent is only reached at the end of the episode
        observation = self._get_obs()
        info = self._get_info()

        return observation, reward, terminated, truncated, info
```

## 注册和创建环境

虽然现在可以立即使用您的新自定义环境，但更常见的做法是使用 {func}`gymnasium.make` 初始化环境。在本节中，我们将解释如何注册自定义环境，然后初始化它。

环境 `ID` 由三个部分组成，其中两个是可选的：可选的名称空间（这里：`gymnasium_env`），必需的名称（这里：`GridWorld`）和可选但推荐的版本（这里：`v0`）。它也可以注册为 `GridWorld-v0` （推荐的方法）、`GridWorld` 或 `gymnasium_env/GridWorld`，然后在环境创建期间应使用适当的 `ID`。

入口点可以是字符串或函数，由于本教程不是 Python 项目的一部分，我们不能使用字符串，但对于大多数环境，这是指定入口点的常用方法。

`Register` 还具有附加参数，可用于指定环境的关键字参数，例如，是否应用时间限制包装器等。有关更多信息，请参阅 {func}`gymnasium.register`。

```python
gym.register(
    id="gymnasium_env/GridWorld-v0",
    entry_point=GridWorldEnv,
)
```

有关注册自定义环境（包括使用字符串入口点）的更完整指南，请阅读完整的[创建环境教程](https://gymnasium.farama.org/tutorials/gymnasium_basics/environment_creation/)。

一旦环境注册完成，您可以通过 {func}`gymnasium.pprint_registry` 进行检查，这将输出所有已注册的环境，然后可以使用 {func}`gymnasium.make` 初始化环境。可以使用 {func}`gymnasium.make_vec` 实例化环境的向量化版本，即并行运行相同环境的多个实例。

```python
import gymnasium as gym
>>> gym.make("gymnasium_env/GridWorld-v0")
<OrderEnforcing<PassiveEnvChecker<GridWorld<gymnasium_env/GridWorld-v0>>>>
>>> gym.make("gymnasium_env/GridWorld-v0", max_episode_steps=100)
<TimeLimit<OrderEnforcing<PassiveEnvChecker<GridWorld<gymnasium_env/GridWorld-v0>>>>>
>>> env = gym.make("gymnasium_env/GridWorld-v0", size=10)
>>> env.unwrapped.size
10
>>> gym.make_vec("gymnasium_env/GridWorld-v0", num_envs=3)
SyncVectorEnv(gymnasium_env/GridWorld-v0, num_envs=3)
```

## 使用包装器

通常，我们想要使用自定义环境的不同变体，或者我们想要修改 Gymnasium 或其他方提供的环境的 `Behavior`。Wrappers 允许我们在不更改环境实现或添加任何样板代码的情况下做到这一点。请查看[包装器文档](https://gymnasium.farama.org/api/wrappers/)，了解如何使用包装器以及有关实现自己的包装器的说明。在我们的示例中，由于观察值是字典，它们不能直接用于学习代码。然而，我们实际上不需要接触我们的环境实现来解决这个问题！我们可以简单地在环境实例的顶部添加一个包装器，将观察值展平为单个数组：

```python
>>> from gymnasium.wrappers import FlattenObservation

>>> env = gym.make('gymnasium_env/GridWorld-v0')
>>> env.observation_space
Dict('agent': Box(0, 4, (2,), int64), 'target': Box(0, 4, (2,), int64))
>>> env.reset()
({'agent': array([4, 1]), 'target': array([2, 4])}, {'distance': 5.0})
>>> wrapped_env = FlattenObservation(env)
>>> wrapped_env.observation_space
Box(0, 4, (4,), int64)
>>> wrapped_env.reset()
(array([3, 0, 2, 1]), {'distance': 2.0})
```